In [45]:
import numpy as np
import scipy as scp
import pandas as pd
from matplotlib import pyplot as plt

In [35]:
# # Pre-processing Section

# path="./ml-latest/"

# # to do preprocessing uncomment the following paragraph and comment the immediate next like to it

# data=pd.read_csv(path+"ratings.csv")
# movie_ids=np.unique(data["movieId"].values)
# max_movie_id=np.max(movie_ids)
# user_ids=np.unique(data["userId"].values)
# max_user_id=np.max(user_ids)
# mat=np.zeros((max_movie_id+1,max_user_id+1))
# for _,i in data.iterrows():
#     mat[int(i[1])][int(i[0])]=i[2]



# Sample Dataset


A_big=np.array(np.random.binomial(n=1,p=0.4,size=(100,100)),dtype=np.float64)
A_small=np.array(np.random.binomial(n=1,p=0.4,size=(50,50)),dtype=np.float64)
A_big=np.ones((100,100))
A_small=np.ones((50,50))



In [36]:
def CUR(A,k):
    Prob_c=np.square(A).sum(axis=0)/np.square(A).sum()
    Prob_r=np.square(A).sum(axis=1)/np.square(A).sum()
    def Column_Select(A,c,flag=True):
        nonlocal Prob_c,Prob_r
        if flag:
            C=np.zeros((A.shape[0],c))
            pos_array=np.zeros(c,dtype=np.int64)
            for i in range(c):
                j=np.random.choice(np.arange(A.shape[1]),p=Prob_c)
                pos_array[i]=j
                C[:,i]=A[:,j]/np.sqrt(c*Prob_c[j])
        else:
            C=np.zeros((c,A.shape[1]))
            pos_array=np.zeros(c,dtype=np.int64)
            for i in range(c):
                j=np.random.choice(np.arange(A.shape[0]),p=Prob_r)
                pos_array[i]=j
                C[i,:]=A[j,:]/np.sqrt(c*Prob_r[j])
        return C,pos_array
    C,col_pos=Column_Select(A,min(4*k,A.shape[1]),True)     # Heuristic
    R,row_pos=Column_Select(A,min(4*k,A.shape[0]),False)    # Heuristic
    ixgrid=np.ix_(row_pos,col_pos)
    W=A[ixgrid]
    U=np.linalg.pinv(W)
    return C,U,R

In [ ]:

U,S,V_T=np.linalg.svd(A_big,full_matrices=False)
S_bar=np.round(S,2)
S_bar=S_bar[S_bar!=0]
num_of_latent=S_bar.shape[0]
print(num_of_latent)

loss_list_big_SVD=[]


for i in range(1,len(A_big)+1):
    A_hat=U[:,:i]@np.diag(S[:i])@V_T[:i,:]
    loss_list_big_SVD.append(np.linalg.norm(A_big-A_hat,ord='fro'))

num_of_latent=0
U,S,V_T=np.linalg.svd(A_small,full_matrices=False)
S_bar=np.round(S,2)
S_bar=S_bar[S_bar!=0]
num_of_latent=S_bar.shape[0]
print(num_of_latent)
loss_list_small_SVD=[]
for i in range(1,len(A_small)+1):
    A_hat=U[:,:i]@np.diag(S[:i])@V_T[:i,:]
    loss_list_small_SVD.append(np.linalg.norm(A_small-A_hat,ord='fro'))

loss_list_big_CUR=[]
for i in range(1,len(A_big)+1):
    C,U,R=CUR(A_big,i)
    A_hat=C@U@R
    loss_list_big_CUR.append(np.linalg.norm(A_big-A_hat,ord='fro'))

loss_list_small_CUR=[]
for i in range(1,len(A_small)+1):
    C,U,R=CUR(A_small,i)
    A_hat=C@U@R
    loss_list_small_CUR.append(np.linalg.norm(A_small-A_hat,ord='fro'))


In [41]:
loss_list_big_CUR=np.array(loss_list_big_CUR)  # O(n)
loss_list_big_SVD=np.array(loss_list_big_SVD)   # O(n)
loss_list_small_SVD=np.array(loss_list_small_SVD)   # O(n)
loss_list_small_CUR=np.array(loss_list_small_CUR)   # O(n)

In [42]:
error_Big=loss_list_big_CUR-loss_list_big_SVD
error_small=loss_list_small_CUR-loss_list_small_SVD


In [50]:
A=np.ones((4,5))
m,n=A.shape

In [ ]:
# P,Q decomposition

A=np.ones((4,5))
def Loss_at(P,Q):
    return 0

def PQ(A,k,eps=0.1):
    m,n=A.shape
    P=np.ones((m,k))
    Q=np.ones((n,k))
    u=5
    for u in range(n):
        Q[u,:]=Q[u,:]-eps*Loss_at(P,Q,A,u)

    return P,Q